# Manutenzione preventiva - 

# Manutenzione preventiva - DoS Attacks

In [21]:
import pandas as pd
import json as json
import numpy as np

In [22]:
clean_cap_30m = open('.\\captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json')
clean_cap_1h = open('.\\captures\\captures1_v2\\clean\\eth2dump-clean-1h_1.json')
data_cc_30m = json.load(clean_cap_30m)
data_cc_1h = json.load(clean_cap_1h)
clean_cap_30m.close()
clean_cap_1h.close()

FileNotFoundError: [Errno 2] No such file or directory: '.\\captures_json\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

In [ ]:
interesting_layers = ["eth", "arp", "icmp", "ip", "tcp", "udp", "mbtcp"]
timestamp_field_name = 'frame.time_epoch'
ipat_field_name = 'inter.packet_arrival_time'
eth_fields = ['eth.dst', 'eth.src', 'eth.type', 'eth.len']
arp_fields = ['arp.hw_type', 'arp.proto_type', 'arp.hw_size', 'arp.proto_size']
icmp_fields = ['icmp.type', 'icmp.code', 'icmp.checksum', 'icmp.checksum_status']
ip_fields = ['ip.version', 'ip.hdr_len', 'ip.dsfield', 'ip.dsfield_dscp', 'ip.dsfield_ecn',
             'ip.len', 'ip.id', 'ip.flags', 'ip.flags_rb', 'ip.flags_df', 'ip.flags_mf',
             'ip.frag_offset', 'ip.ttl', 'ip.proto', 'ip.checksum', 'ip.checksum_status', 'ip.src', 'ip.dst']
tcp_fields = ['tcp.srcport', 'tcp.dstport', 'tcp.stream', 'tcp.len', 'tcp.seq', 'tcp.nxtseq', 'tcp.ack',
              'tcp.hdr_len', 'tcp.flags', 'tcp.flags_res', 'tcp.flags_ae', 'tcp.flags_cwr', 'tcp.flags_ece',
              'tcp.flags_urg', 'tcp.flags_ack', 'tcp.flags_push', 'tcp.flags_reset', 'tcp.flags_syn', 'tcp.flags_fin',
              'tcp.flags_str', 'tcp.window_size_value', 'tcp.window_size', 'tcp.window_size_scalefactor', 'tcp.checksum',
              'tcp.checksum_status', 'tcp.urgent_pointer']
udp_fields = ['udp.srcport', 'udp.dstport', 'udp.port', 'udp.length', 'udp.checksum', 'udp.checksum_status']
mbtcp_fields = ['mbtcp.trans_id', 'mbtcp.prot_id', 'mbtcp.len', 'mbtcp.unit_id']
interesting_layer_fields = [eth_fields, arp_fields, icmp_fields, ip_fields, tcp_fields, udp_fields, mbtcp_fields]
df_columns = [timestamp_field_name] + eth_fields + arp_fields + icmp_fields + ip_fields + tcp_fields + udp_fields + mbtcp_fields
packets_values = []

In [ ]:
counter = 0
for packet in data_cc_30m:
    tmp_pkt = []

    tmp_pkt.append(packet['_source']['layers']['frame']['frame.time_epoch'])
    
    for i in range(len(interesting_layers)):
        if interesting_layers[i] in packet['_source']['layers']:
            for field in interesting_layer_fields[i]:
                if field in packet['_source']['layers'][interesting_layers[i]]:
                    tmp_pkt.append(packet['_source']['layers'][interesting_layers[i]][field])
                else:
                    tmp_pkt.append(np.nan)
        else:
            for field in interesting_layer_fields[i]:
                tmp_pkt.append(np.nan)
    packets_values.append(tmp_pkt)

print(len(packets_values[1]))
print(len(packets_values))
      

In [ ]:
df_clean_30m = pd.DataFrame(data=packets_values, columns=df_columns)
df_clean_30m.head()